In [ ]:
import logging
import fastf1
import pandas as pd
import numpy as np

logging.getLogger('fastf1').setLevel(logging.CRITICAL)
fastf1.Cache.enable_cache('cache')

# --- CONFIGURATION ---
season_range = {2022, 2023}
races_per_season = {2022: 21, 2023: 22}

# Safety Car & VSC codes
SC_STATUS_CODES = {4, 6}

# Status codes to remove from Status column AFTER labeling
CODES_TO_REMOVE_FROM_STATUS = {4, 6, 7}  # 4=SC, 6=VSC, 7=SC ending

# Flag and warning codes
WARNING_STATUS_CODES = {2, 5}
# ---------------------


all_laps = []
all_weather = []
all_trackStatus = []

for season in season_range:
    for round_num in range(1, races_per_season[season] + 1):
        print(f"Loading Season {season} Round {round_num}")
        try:
            session = fastf1.get_session(season, round_num, "R")
            session.load(laps=True, telemetry=False, weather=True, messages=True)

            laps = session.laps.copy()
            laps["Season"] = season
            laps["Round"] = round_num
            all_laps.append(laps)

            weather = session.weather_data.copy()
            weather["Season"] = season
            weather["Round"] = round_num
            all_weather.append(weather)

            trackStatus = session.track_status.copy()
            trackStatus["Season"] = season
            trackStatus["Round"] = round_num
            all_trackStatus.append(trackStatus)

        except Exception as e:
            print(f"Skipping Season {season} Round {round_num} due to error: {e}")

# Combine objects
laps_all = pd.concat(all_laps, ignore_index=True)
laps_all = laps_all[['LapStartTime', 'Season', 'Round', 'Driver', 'LapNumber',
                     'LapTime', 'TyreLife', 'Sector1Time', 'Sector2Time', 'Sector3Time']]

weather_all = pd.concat(all_weather, ignore_index=True)
trackStatus_all = pd.concat(all_trackStatus, ignore_index=True)

# Clean nulls
laps_all = laps_all.dropna(subset=['LapStartTime'])
weather_all = weather_all.dropna(subset=['Time'])
trackStatus_all = trackStatus_all.dropna(subset=['Time'])

# Sort
laps_all = laps_all.sort_values(by=['Season', 'Round', 'LapStartTime'])
weather_all = weather_all.sort_values(by=['Season', 'Round', 'Time'])
trackStatus_all = trackStatus_all.sort_values(by=['Season', 'Round', 'Time'])

# Merge data
merged_data = pd.merge_asof(
    laps_all,
    weather_all,
    left_on='LapStartTime',
    right_on='Time',
    by=['Season', 'Round'],
    direction='nearest'
)

merged_data = pd.merge_asof(
    merged_data,
    trackStatus_all,
    left_on='LapStartTime',
    right_on='Time',
    by=['Season', 'Round'],
    direction='nearest'
)

merged_data.reset_index(drop=True, inplace=True)

# Clean Status column
merged_data['Status'] = (
    merged_data['Status']
    .astype(str)
    .str.replace(r'\D', '', regex=True)
    .replace('', np.nan)
    .fillna(1)
    .astype(int)
)

print("Status values before cleaning:", merged_data['Status'].unique())


# --- 1. CREATE TARGET FOR FIRST SC/VSC OCCURRENCE ONLY ---
def create_first_occurrence_label(group):
    group = group.sort_values(by='LapStartTime').reset_index(drop=True)

    # Identify SC/VSC laps
    group['is_sc'] = group['Status'].apply(lambda x: 1 if x in SC_STATUS_CODES else 0)

    # Find first SC lap index (if none exist, return all zeros)
    sc_indices = group.index[group['is_sc'] == 1]

    if len(sc_indices) == 0:
        group['target'] = 0
        return group

    first_sc_index = sc_indices[0]

    # Label lap BEFORE first SC as 1 (if exists)
    target_index = first_sc_index - 1
    group['target'] = 0
    if target_index >= 0:
        group.loc[target_index, 'target'] = 1

    return group


# Apply per race
merged_data = merged_data.groupby(['Season', 'Round'], group_keys=False).apply(create_first_occurrence_label)


# --- 2. CLEAN STATUS (REMOVE 4, 6, 7 FOR FEATURES) ---
merged_data['Status'] = merged_data['Status'].apply(
    lambda x: 1 if x in CODES_TO_REMOVE_FROM_STATUS else x
)

print("Status values AFTER cleaning:", merged_data['Status'].unique())


# --- 3. TRACK WARNING FEATURE ---
merged_data['TrackWarning'] = merged_data['Status'].apply(
    lambda x: 1 if x in WARNING_STATUS_CODES else 0
)

# --- 4. REMOVE LABELING UTILS ---
merged_data = merged_data.drop(columns=['is_sc', 'Message'], errors='ignore')


# Summary
print("\nFinal target distribution:")
print(merged_data['target'].value_counts())

# Save
merged_data.to_csv('merged_data_first_occurrence_target.csv', index=False)
print("\nSaved as 'merged_data_first_occurrence_target.csv'")


Loading Season 2023 Round 1
Loading Season 2023 Round 2
Loading Season 2023 Round 3


In [ ]:
import fastf1
import pandas as pd

fastf1.Cache.enable_cache('cache')

year = 2023
round_num = 1

print(f"Loading session: {year}, Round {round_num}, Race")
session = fastf1.get_session(year, round_num, 'R')
session.load(laps=True, telemetry=True, weather=True)
print("Session data loaded.")

laps = session.laps.copy()
weather = session.weather_data.copy()
track_status = session.track_status.copy()  # <-- NEW

# Normalize column names for track_status
track_status.columns = [col.strip() for col in track_status.columns]
if 'Status' in track_status.columns:
    track_status.rename(columns={'Status': 'TrackStatus'}, inplace=True)
elif 'TrackStatus' not in track_status.columns:
    # Fallback — create an empty TrackStatus column if missing
    track_status['TrackStatus'] = None

all_laps_data = pd.DataFrame()

for i, lap in laps.iterrows():
    try:
        lap_telem = lap.get_telemetry()
    except Exception as e:
        print(f"Warning: Could not get telemetry for lap {lap['LapNumber']} by {lap['Driver']}. Error: {e}")
        continue

    if not lap_telem.empty:
        # Merge weather
        if not weather.empty:
            lap_telem = pd.merge_asof(
                lap_telem.sort_values('SessionTime'),
                weather.sort_values('Time'),
                left_on='SessionTime',
                right_on='Time',
                direction='backward',
                suffixes=('_telem', '_weather')
            )

        # Merge track status
        if not track_status.empty:
            lap_telem = pd.merge_asof(
                lap_telem.sort_values('SessionTime'),
                track_status.sort_values('Time'),
                left_on='SessionTime',
                right_on='Time',
                direction='backward'
            )

        # Add lap info
        lap_telem['Driver'] = lap['Driver']
        lap_telem['LapNumber'] = lap['LapNumber']
        lap_telem['LapTime'] = lap['LapTime']
        lap_telem['Sector1Time'] = lap['Sector1Time']
        lap_telem['Sector2Time'] = lap['Sector2Time']
        lap_telem['Sector3Time'] = lap['Sector3Time']

        if 'Compound' in lap:
            lap_telem['TyreCompound'] = lap['Compound']
        if 'TyreLife' in lap:
            lap_telem['TyreLife'] = lap['TyreLife']

        lap_telem = lap_telem.set_index('SessionTime')

        # Create aggregation dictionary dynamically
        agg_dict = {
    'Driver': 'first',
    'LapNumber': 'first',
    'LapTime': 'first',
    'Sector1Time': 'first',
    'Sector2Time': 'first',
    'Sector3Time': 'first',
    'Distance': 'max',
    'RelativeDistance': 'max',
    'TyreLife': 'first',
    'Speed': 'mean',
    'Throttle': 'mean',
    'Brake': 'max',
    'AirTemp': 'mean',
    'Humidity': 'mean',
    'Pressure': 'mean',
    'TrackTemp': 'mean',
    'WindDirection': 'mean',
    'WindSpeed': 'mean'
}

if 'TrackStatus' in lap_telem.columns:
    agg_dict['TrackStatus'] = 'last'

lap_telem_resampled = lap_telem.resample('3S').agg(agg_dict)

# Then separately compute rainfall min & max
rainfall_stats = lap_telem.resample('3S')['Rainfall'].agg(
    RainfallMin='min',
    RainfallMax='max'
)

# Combine them together
lap_telem_resampled = lap_telem_resampled.join(rainfall_stats)

lap_telem_resampled = lap_telem_resampled.dropna(how='all')

all_laps_data = pd.concat([all_laps_data, lap_telem_resampled])

# Reset and reorder columns
all_laps_data = all_laps_data.reset_index()
cols = all_laps_data.columns.tolist()
cols.insert(0, cols.pop(cols.index('Driver')))
all_laps_data = all_laps_data[cols]

all_laps_data.to_csv('all_drivers_lap_telem_with_trackstatus.csv', index=False)
print("✅ Data saved to 'all_drivers_lap_telem_with_trackstatus.csv'.")


Loading session: 2023, Round 1, Race


core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '14', '55', '44', '18', '63', '77', '10', '23', '22', '2', '20', '21', '27', '24', '4', '31', '16', '81']


Session data loaded.


/tmp/ipykernel_163496/623046569.py:99: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  lap_telem_resampled = lap_telem.resample('3S').agg(agg_dict).dropna(how='all')
/tmp/ipykernel_163496/623046569.py:99: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  lap_telem_resampled = lap_telem.resample('3S').agg(agg_dict).dropna(how='all')
/tmp/ipykernel_163496/623046569.py:99: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  lap_telem_resampled = lap_telem.resample('3S').agg(agg_dict).dropna(how='all')
/tmp/ipykernel_163496/623046569.py:99: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  lap_telem_resampled = lap_telem.resample('3S').agg(agg_dict).dropna(how='all')
/tmp/ipykernel_163496/623046569.py:99: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' 

✅ Data saved to 'all_drivers_lap_telem_with_trackstatus.csv'.


In [18]:
import fastf1
import pandas as pd

# Enable cache
fastf1.Cache.enable_cache('cache')

year = 2023
round_num = 1

print(f"Loading session: {year}, Round {round_num}, Race")
session = fastf1.get_session(year, round_num, 'R')
session.load(laps=True, telemetry=True, weather=True)
print("Session data loaded.")

laps = session.laps.copy()
weather = session.weather_data.copy()
track_status = session.track_status.copy()  # <-- NEW

# Normalize column names for track_status
track_status.columns = [col.strip() for col in track_status.columns]
if 'Status' in track_status.columns:
    track_status.rename(columns={'Status': 'TrackStatus'}, inplace=True)
elif 'TrackStatus' not in track_status.columns:
    # Fallback — create an empty TrackStatus column if missing
    track_status['TrackStatus'] = None

all_laps_data = pd.DataFrame()

for i, lap in laps.iterrows():
    try:
        lap_telem = lap.get_telemetry()
    except Exception as e:
        print(f"Warning: Could not get telemetry for lap {lap['LapNumber']} by {lap['Driver']}. Error: {e}")
        continue

    if not lap_telem.empty:
        # Merge weather
        if not weather.empty:
            lap_telem = pd.merge_asof(
                lap_telem.sort_values('SessionTime'),
                weather.sort_values('Time'),
                left_on='SessionTime',
                right_on='Time',
                direction='backward',
                suffixes=('_telem', '_weather')
            )

        # Merge track status
        if not track_status.empty:
            lap_telem = pd.merge_asof(
                lap_telem.sort_values('SessionTime'),
                track_status.sort_values('Time'),
                left_on='SessionTime',
                right_on='Time',
                direction='backward'
            )

        # Add lap info
        lap_telem['Driver'] = lap['Driver']
        lap_telem['LapNumber'] = lap['LapNumber']
        lap_telem['LapTime'] = lap['LapTime']
        lap_telem['Sector1Time'] = lap['Sector1Time']
        lap_telem['Sector2Time'] = lap['Sector2Time']
        lap_telem['Sector3Time'] = lap['Sector3Time']

        if 'Compound' in lap:
            lap_telem['TyreCompound'] = lap['Compound']
        if 'TyreLife' in lap:
            lap_telem['TyreLife'] = lap['TyreLife']

        lap_telem = lap_telem.set_index('SessionTime')

        # Base aggregation dictionary
        agg_dict = {
            'Driver': 'first',
            'LapNumber': 'first',
            'LapTime': 'first',
            'Sector1Time': 'first',
            'Sector2Time': 'first',
            'Sector3Time': 'first',
            'Distance': 'max',
            'RelativeDistance': 'max',
            'TyreLife': 'first',
            'Speed': 'mean',
            'Throttle': 'mean',
            'Brake': 'max',
            'AirTemp': 'mean',
            'Humidity': 'mean',
            'Pressure': 'mean',
            'TrackTemp': 'mean',
            'WindDirection': 'mean',
            'WindSpeed': 'mean'
        }

        # Only include TrackStatus if it exists
        if 'TrackStatus' in lap_telem.columns:
            agg_dict['TrackStatus'] = 'last'

        # Aggregate main telemetry
        lap_telem_resampled = lap_telem.resample('3S').agg(agg_dict)

        # Compute Rainfall min and max separately
        if 'Rainfall' in lap_telem.columns:
            rainfall_stats = lap_telem.resample('3S')['Rainfall'].agg(
                RainfallMin='min',
                RainfallMax='max'
            )
            # Join back to main DataFrame
            lap_telem_resampled = lap_telem_resampled.join(rainfall_stats)

        lap_telem_resampled = lap_telem_resampled.dropna(how='all')

        all_laps_data = pd.concat([all_laps_data, lap_telem_resampled])

# Reset and reorder columns
all_laps_data = all_laps_data.reset_index()
cols = all_laps_data.columns.tolist()
cols.insert(0, cols.pop(cols.index('Driver')))
all_laps_data = all_laps_data[cols]

# Save final dataset
all_laps_data.to_csv('all_drivers_lap_telem_with_trackstatus.csv', index=False)
print("✅ Data saved to 'all_drivers_lap_telem_with_trackstatus.csv'.")


Loading session: 2023, Round 1, Race


core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '14', '55', '44', '18', '63', '77', '10', '23', '22', '2', '20', '21', '27', '24', '4', '31', '16', '81']
/tmp/ipykernel_163496/666521201.py:100: FutureWarning: 'S' is

Session data loaded.


/tmp/ipykernel_163496/666521201.py:100: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  lap_telem_resampled = lap_telem.resample('3S').agg(agg_dict)
/tmp/ipykernel_163496/666521201.py:104: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  rainfall_stats = lap_telem.resample('3S')['Rainfall'].agg(
/tmp/ipykernel_163496/666521201.py:100: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  lap_telem_resampled = lap_telem.resample('3S').agg(agg_dict)
/tmp/ipykernel_163496/666521201.py:104: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  rainfall_stats = lap_telem.resample('3S')['Rainfall'].agg(
/tmp/ipykernel_163496/666521201.py:100: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  lap_telem_resampled = lap_telem.resample('3S').agg(agg_dict)

✅ Data saved to 'all_drivers_lap_telem_with_trackstatus.csv'.
